# Model - Matrix Factorization


- Map both users and items to a joint latent factor space
- user-item interactions are inner products in that space

**Decomposition of the rating matrix**  
R -> m users, n items  
Row in P -> user's affinity to the features (m users, f latent factors/features)  
Row in Q -> item's relation to the features (n items, f latent factors/features)  

- P indicates how much user likes f latent factors  
- Q indicates how much one item obtains f latent factors
- The dot product indicates how much user likes item  
- The decomposition automatically ranks features by their impact on the ratings
- Features may not be intuitive though !
- Model has hyperparameters (regularization, learning rate)

**Common Learning Algorithms**

- stochastic gradient descent
- alternating least squares

**Key Challenge**

Temporal Dynamics.




For a better and visual understanding of Matrix Factorization Techniques, read the following links
- The original paper on [Collaborative Filtering for Implicit Feedback Datasets](http://yifanhu.net/PUB/cf.pdf)
- [Finding Similar Music using Matrix Factorization](http://www.benfrederickson.com/matrix-factorization/) by Ben Frederickson
- [Intro to Implicit Matrix Factorization: Classic ALS with Sketchfab Models](http://blog.ethanrosenthal.com/2016/10/19/implicit-mf-part-1/)


## Simple Example

In [17]:
import numpy as np
np.set_printoptions(precision=2)

In [38]:
"""
@INPUT:
    R     : a matrix to be factorized, dimension N x M
    P     : an initial matrix of dimension N x K
    Q     : an initial matrix of dimension M x K
    K     : the number of latent features
    steps : the maximum number of steps to perform the optimisation
    alpha : the learning rate
    beta  : the regularization parameter
@OUTPUT:
    the final matrices P and Q
"""

def als_matrix_factorization(R, K=2, steps=5000, alpha=0.0002, beta=0.02):
    N = len(R)
    M = len(R[0])
    P = np.random.rand(N,K)
    Q = np.random.rand(M,K)
    Q = Q.T
    
    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = np.dot(P,Q)
        e = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
        if e < 0.001:
            break
    return P, Q.T

In [39]:
R = np.array([[1, 0, 1, 0, 0], 
              [0, 1, 0, 0, 1], 
              [1, 0, 0, 1, 0], 
              [0, 0, 1, 0, 0]])

In [40]:
# Run ALS
nP, nQ = als_matrix_factorization(R, K=2)

In [41]:
nP.dot(nQ.T)

array([[ 0.97,  0.97,  0.99,  0.85,  1.03],
       [ 0.97,  0.96,  0.98,  0.84,  1.01],
       [ 1.01,  1.02,  1.09,  0.98,  1.13],
       [ 0.98,  0.97,  0.99,  0.85,  1.03]])

In [32]:
nQ

array([[ 0.83,  0.68],
       [ 0.78,  0.45],
       [ 0.92,  0.78],
       [ 1.03,  0.26],
       [ 0.75,  0.35]])

In [42]:
nP

array([[ 0.5 ,  0.86],
       [ 0.49,  0.85],
       [ 0.61,  0.87],
       [ 0.49,  0.86]])

## HackerNews Dataset

In [57]:
# ! pip install implicit

In [58]:
import pandas as pd
import scipy.sparse as sparse
import implicit

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [59]:
# Lets read the story, user, comments data
story_user_comments = pd.read_csv("data/story_user_comment.csv")

In [60]:
story_user_comments.head()

,user,story,comment
0,21,14356377,1
1,21,15131370,1
2,21,15196309,1
3,47,15601729,1
4,47,14023198,1


In [61]:
n_users = story_user_comments.user.unique().shape[0]
n_stories = story_user_comments.story.unique().shape[0]

print('Number of users: {}'.format(n_users))
print('Number of stories: {}'.format(n_stories))
print('Sparsity: {:4.3f}%'.format(float(story_user_comments.shape[0]) / float(n_users*n_stories)))

Number of users: 23230
Number of stories: 969
Sparsity: 0.002%


## Creating the user_story matrix

In [97]:
#! pip install mlxtend

In [98]:
from mlxtend.preprocessing import OnehotTransactions

In [99]:
def OHE_Matrix( df ) :

    g2 = df.groupby(["user"], as_index = False)
    
    Itemset = []
    user = []
    for item in list(g2.groups.keys()) :
        Itemset.append( list(g2.get_group(item)["story"]))
        user.append(item) 
        
    oht = OnehotTransactions()
    u = oht.fit(Itemset).transform(Itemset)
    
    Matrix = pd.DataFrame(u, columns = oht.columns_)
    Matrix["user"] = user
    Matrix = Matrix.set_index("user")
    
    return Matrix

In [1]:
matrix1 = OHE_Matrix(story_user_comments)

NameError: name 'OHE_Matrix' is not defined

**Using Sparse Matrix Creation**

Lets create a mapper to convert this to a sparse matrix

In [62]:
def Sparse_Matrix(df):
    
    # Create mappings
    user_to_index = {}
    index_to_user = {}
    for (index, user) in enumerate(df.user.unique().tolist()):
        user_to_index[user] = index
        index_to_user[index] = user

    story_to_index = {}
    index_to_story = {}
    for (index, story) in enumerate(df.story.unique().tolist()):
        story_to_index[story] = index
        index_to_story[index] = story
        
    # Create a map id function
    def map_ids(row, mapper):
        return mapper[row]
    
    # Apply the map id function 
    I = df.user.apply(map_ids, args=[user_to_index]).as_matrix()
    J = df.story.apply(map_ids, args=[story_to_index]).as_matrix()
    V = np.ones(I.shape[0])
    
    # Create the Matrix
    item_user = sparse.coo_matrix((V, (I, J)), dtype=np.float64)
    story_user_matrix = df.tocsr()

In [91]:
# Create the Matrix
story_user_matrix = sparse.coo_matrix((V, (I, J)), dtype=np.float64)
story_user_matrix = story_user_matrix.tocsr()

In [87]:
story_user_matrix

<23230x969 sparse matrix of type '<class 'numpy.float64'>'
	with 50975 stored elements in Compressed Sparse Row format>

## Building the Recommendation

In [71]:
import implicit

In [73]:
# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=10)

In [88]:
# train the model on a sparse matrix of item/user/weights
model.fit(story_user_matrix)

In [90]:
# recommend items for a user
user_items = story_user_matrix.T.tocsr()

In [ ]:
recommendations = model.recommend(user, implicit_mat)

In [ ]:
# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=50)

# train the model on a sparse matrix of item/user/confidence weights
model.fit(implicit_mat)

# recommend items for a user
user_items = item_user_data.T.tocsr()
recommendations = model.recommend(userid, user_items)

# find related items
related = model.similar_items(itemid)